In [1]:
import malaya

In [2]:
bert = malaya.sentiment.transformer(model = 'bert')
xlnet = malaya.sentiment.transformer(model = 'xlnet')
albert = malaya.sentiment.transformer(model = 'albert')

In [3]:
import numpy as np
import json

In [4]:
with open('ps.json') as fopen:
    ps = json.load(fopen)
    
len(ps)

1529321

In [5]:
from tqdm import tqdm

batch_size = 128

results = []
for i in tqdm(range(0, len(ps), batch_size)):
    index = min(i + batch_size, len(ps))
    batch = ps[i: index]
    bert_r = bert.predict_batch(batch,get_proba = True,add_neutral=False)
    xlnet_r = xlnet.predict_batch(batch,get_proba = True,add_neutral=False)
    albert_r = albert.predict_batch(batch,get_proba = True,add_neutral=False)
    bert_r = np.array([list(r.values()) for r in bert_r])
    xlnet_r = np.array([list(r.values()) for r in xlnet_r])
    albert_r = np.array([list(r.values()) for r in albert_r])
    results.append((bert_r + xlnet_r + albert_r) / 3)

100%|██████████| 11948/11948 [2:07:03<00:00,  1.57it/s] 


In [6]:
results = np.concatenate(results, axis = 0)
results.shape

(1529321, 2)

In [7]:
Y = [1] * (results.shape[0] - 1) + [0]

In [8]:
numpy_array_of_noisy_labels = np.array(Y)
results.shape, numpy_array_of_noisy_labels.shape 

((1529321, 2), (1529321,))

In [9]:
from cleanlab.pruning import get_noise_indices

ordered_label_errors = get_noise_indices(
    s = numpy_array_of_noisy_labels,
    psx = results,
    sorted_index_method='normalized_margin',
 )

In [10]:
ordered_label_errors.shape

(444729,)

In [11]:
for i in ordered_label_errors[:1000]:
    if np.argmax(results[i]) == 0:
        print(results[i][0], ps[i].replace('\n', '. '))

0.9999239 @Iwwannnnn apa lancau mintak izin kat awek . aku nak keluar rumah mak aku tak pernah halang pun . betul bini ye perempuan tu
0.9999199 Mas Bowo di marahin tukang parkir!. . "Ancene capres 02 cangkeme ora tau di toto?  . . Tukang Parkir Kesal dan menganca https://t.co/hWuTPaWBP0
0.99991864 Ala bodohnya jilat. Gersang sgt ka. Geli aku 
0.99991864 @lalajoeee Si lancau ni mmg kuat cari pasal. Bukan dgn aku je. Dari hari tu ade gak org lain dia cari pasal
0.999918 Eyh babila. Dah pandai masok umah tu tutup la pintu shibei. Ajar seribu kali pon tak faham lagi
0.9999179 Ilmu gratis tapi bukan berarti boleh ngebodoh bodohin,nih paling banyak perkara kalo ke gym bawa temen yg masi besa https://t.co/d5rjwpxsil
0.99991703 Makin melampo mintak murah !! Butoh !! Dah nak mcam harga trade in kt kedaiii , lancau la badawi
0.99991685 @ainohyeah Amboi marahnya.  Tapi durian tu nampak sedap.
0.9999168 SI ADAM NI NAFSU KUDA AKU RASA?!!! HA RETI PULAK ALLAHUAKHBAR?!!!!!! BABIKKKKKK!!. . DIAMLAH H

In [12]:
not_in = set(np.arange(len(ps))) - set(ordered_label_errors)

strong_positives = []
for i in tqdm(not_in):
    strong_positives.append(ps[i])

100%|██████████| 1084592/1084592 [00:00<00:00, 1874368.23it/s]


In [17]:
len(strong_positives)

1084592

In [14]:
with open('strong_positives.json', 'w') as fopen:
    json.dump(strong_positives, fopen)

In [15]:
weak_negatives = []
for i in tqdm(ordered_label_errors):
    weak_negatives.append(ps[i])

100%|██████████| 444729/444729 [00:00<00:00, 1438246.95it/s]


In [16]:
with open('weak-negatives.json', 'w') as fopen:
    json.dump(weak_negatives, fopen)